python setup.py sdist bdist_wheel

twine upload --skip-existing dist/*

In [1]:
import pandas as pd
import ScraperFC as sfc
import traceback

In [6]:
scraper = sfc.Capology()
try:
    out = scraper.scrape_salaries(year=2022, league='La Liga', currency='eur')
except:
    traceback.print_exc()
finally:
    scraper.close()

In [7]:
out

,Player,Weekly Gross,Annual Gross,Expiration,Length,Total Gross,Status,Pos. group,Pos.,Age,Country,Club
0,Antoine Griezmann,"€ 721,154","€ 37,500,000","Jun 30, 2022",1-yr,"€ 37,500,000",Starter,F,SS,30,France,Atletico Madrid
1,Eden Hazard,"€ 540,962","€ 28,130,000","Jun 30, 2024",3-yrs,"€ 84,390,000",Reserve,F,LW,30,Belgium,Real Madrid
2,Gareth Bale,"€ 540,962","€ 28,130,000","Jun 30, 2022",1-yr,"€ 28,130,000",Reserve,F,RW,32,Wales,Real Madrid
3,David Alaba,"€ 432,692","€ 22,500,000","Jun 30, 2026",5-yrs,"€ 112,500,000",Starter,D,CB,29,Austria,Real Madrid
4,Sergio Busquets,"€ 423,077","€ 22,000,000","Jun 30, 2023",2-yrs,"€ 44,000,000",Starter,M,DM,33,Spain,Barcelona
...,...,...,...,...,...,...,...,...,...,...,...,...
505,Javi López,€ 769,"€ 40,000","Jun 30, 2023",2-yrs,"€ 80,000",Reserve,D,LB,19,Spain,Alaves
506,Diego Conde,€ 577,"€ 30,000","Jun 30, 2024",3-yrs,"€ 90,000",Reserve,K,GK,22,Spain,Getafe
507,Rober González,€ 385,"€ 20,000","Jun 30, 2024",3-yrs,"€ 60,000",Reserve,F,RW,20,Spain,Real Betis
508,Josete Miranda,€ 0,€ 0,"Jun 30, 2022",1-yr,€ 0,Reserve,F,LW,23,Equatorial Guinea,Getafe
